In [15]:
import pandas as pd
import os
from openai import OpenAI
from dotenv import load_dotenv

from crewai import Agent, Task, Crew, Process
# from crewai_tools import SerperDevTool, FileReadTool, ScrapeWebsiteTool, WebsiteSearchTool
from crewai_tools import SerperDevTool, ScrapeWebsiteTool

In [16]:
load_dotenv('.env')
os.environ['OPENAI_MODEL_NAME'] = "gpt-4o-mini"
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
SERPER_API_KEY = os.getenv("SERPER_API_KEY")
client = OpenAI(api_key=OPENAI_API_KEY)

In [17]:
topic = input("Enter the audit topic: ")
if topic:
    print("You entered:", topic)
else:
    print("Please enter an audit task.")

You entered: post secondary education account


In [18]:
tool_search = SerperDevTool(n_results=5, country="sg")
tool_webscrape = ScrapeWebsiteTool()

auditor = Agent(
    role = "auditor",
    goal="Return the Audit Planning Memo on {topic} based on inputs from...",
    backstory = """You are an auditor. You need to consider the inputs from the researcher to determine what to write for the audit planning memo and risk assessment""",
    allow_delegation = True,
    max_iter = 10,
    max_execution_time = 30,
    verbose = True
)

researcher = Agent(
    role="researcher",
    goal="From the internet, research and analyse and sieve out information related to {topic} only." ,
    backstory="""As a researcher, navigating and extracting critical information is crucial. You are assisting the auditor to research on topics relevant to {topic}. 
    """,
    allow_delegation=False,
    verbose=True,
    max_iter = 10,
    max_execution_time = 30,
    max_rpm = 5
)


task_search = Task(
    description="""Returns a JSON list of websites that contains the top 5 most relevant information, in the context of Singapore, that is related to policy and regulation on {topic} only. Ignore urls that ends with .pdf.
    """,
    expected_output="""\
    Returns websites that contain the top 5 most relavant policy and regulation information on {topic} only. The output must be in JSON format with the description as the key and url as value.""",

    agent=researcher,
    tools=[tool_search],
)


task_websitesearch = Task(
    description="""Scrape through a website for information related to {topic} only
    """,
    expected_output="""\
    Summarise the website for information related to {topic} only in the Singapore context. """,

    agent=researcher,
    tools=[tool_webscrape],
    context = [task_search]
)

task_write= Task(
    description="""\
    1. Use the content to craft an audit planning memo and risk assessment based on {topic} .
    2. Sections are properly named in an engaging manner.
    3. Proofread for grammatical errors and alignment with the common style used in audit planning memos.""",

    expected_output="""
    A well-written audit planning memo and risk assessment in markdown format. Each section should have minimum of 2 and maximum of 5 paragraphs.""",
    agent=auditor,
    context=[task_websitesearch],
    output_file="APM.md"
)




In [19]:
#creating the crew

crew = Crew(
    agents=[auditor,researcher],
    tasks=[task_search,task_websitesearch,task_write],
    verbose=True
)


2024-10-08 09:52:27,962 - 11040 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


In [20]:
result = crew.kickoff(inputs={"topic": topic})

 [2024-10-08 09:52:32][DEBUG]: == Working Agent: researcher
 [2024-10-08 09:52:32][INFO]: == Starting Task: Returns a JSON list of websites that contains the top 5 most relevant information, in the context of Singapore, that is related to policy and regulation on post secondary education account only. Ignore urls that ends with .pdf.
    


> Entering new CrewAgentExecutor chain...
I need to search for information related to policy and regulation on post-secondary education accounts in Singapore. This will help in compiling a list of relevant websites.

Action: Search the internet  
Action Input: {"search_query": "policy regulation post secondary education account Singapore"}  
 


Search results: Title: Post-Secondary Education Account (PSEA) - MOE
Link: https://www.moe.gov.sg/financial-matters/psea
Snippet: The Post-Secondary Education Account (PSEA) is part of the Post-Secondary Education Scheme to help Singapore Citizens pay for their post-secondary education.
---
Title: Post-Secon

In [21]:
print(f"Raw Output: {result.raw}")
print(f"Token Usage: {result.token_usage}")
print("-----------------------------------------\n\n")
print(f"Tasks Output of Task 1: {result.tasks_output[0]}")
print("-----------------------------------------\n\n")

Raw Output: # Audit Planning Memo and Risk Assessment for Post-Secondary Education Account

## Introduction

This audit planning memo outlines the key risks and considerations relevant to the post-secondary education account, which encompasses various financial, compliance, operational, and reputational factors. The objective of this audit is to ensure the integrity of financial reporting, adherence to regulatory requirements, and the effectiveness of operational processes within the institution. By identifying and assessing these risks, the audit team can develop a strategic approach to the audit process.

## Financial Risks

### Revenue Recognition

One significant financial risk within post-secondary institutions is the potential for misstatements in revenue recognition. These institutions typically generate revenue from diverse sources, including tuition fees, grants, and donations. The complexities of funding arrangements, particularly those involving federal and state financial a

In [ ]:
# tool = SerperDevTool(n_results=3, country="sg")

# agent = Agent(
#     role="search agent",
#     goal="return most relevant information, in the context of Singapore, that is related to policy and regulation on {topic}" ,
#     backstory="""You're working on gathering all relavant policy and regualtion information on {topic}.
#     """,
#     allow_delegation=False,
#     verbose=True,
#     max_iter=40
# )

# task = Task(
#     description="""return most relevant information, in the context of Singapore, that is related to policy and regulation on {topic}
#     """,
#     expected_output="""\
#     A detailed report with most relevant information information that is related to policy and regulation on {topic}.""",

#     agent=agent,
#     tools=[tool],
# )

# crew = Crew(
#     agents=[agent],
#     tasks=[task],
#     verbose=True
# )

# result = crew.kickoff(inputs={"topic": topic})

In [ ]:
# print(f"Raw Output: {result.raw}")
# print(f"Token Usage: {result.token_usage}")
# print("-----------------------------------------\n\n")
# print(f"Tasks Output of Task 1: {result.tasks_output[0]}")
# print("-----------------------------------------\n\n")
# # print(f"Tasks Output of Task 2: {result.tasks_output[1]}")